# Full Implementation of Neural Network

#### Import Libraries

In [16]:
import scipy.io
import numpy as np
from scipy.optimize import fmin_cg
import pickle

#### Load weights to verify cost function

In [4]:
mat = scipy.io.loadmat('ex4weights.mat')

#### Retrieving parameters for the input and hidden layers

In [5]:
Theta1 = mat['Theta1']
Theta2 = mat['Theta2']

### Loading the DataSet

In [6]:
data = scipy.io.loadmat('ex4data1.mat')

#### Getting X and y matrices

In [56]:
X = data['X']
y = data['y']

# Shuffle them in random order (only rows)
# to make model more flexible
np.random.shuffle(X)
np.random.shuffle(y)

### Splitting the dataset to train_set and test_set

In [57]:
m_train = 4000
m_test = 1000

X_train = X[0:4000]
y_train = y[0:4000]

X_test = X[4000:]
y_test = y[4000:]

### Unroll parameters

In [7]:
print(np.shape(Theta1), 25*401)
print(np.shape(Theta2), 10*26)

(25, 401) 10025
(10, 26) 260


In [8]:
nn_params = np.concatenate([Theta1.flatten(), Theta2.flatten()])
np.shape(nn_params)

(10285,)

### Rollback

In [9]:
Theta1 = np.reshape(nn_params[0:10025], (-1, 401))
Theta2 = np.reshape(nn_params[10025:10285], (-1, 26))

#### Setup some useful variables

In [9]:
m = np.shape(X)[0] # number of  examples
n = np.shape(X)[1] # number of features
k = 10 # number of labels
J = 0 # Cost function value

## Implementing the cost function

#### Converting y( y(i) -> 1:10) to Y( Y(i) -> 0, 1)

In [11]:
# Y = matrix of where each row(i) 
# represents corresponding X(i) example
# and each column correlates to output's classes
y[y == 0] = 10

Y = np.zeros([m, k])
for i in range(m):
    j = y[i]
    Y[i, j-1] = 1

#### Implementing sigmoid function

In [24]:
def sigmoid(z):
    #return 1.0 / (1.0 + np.exp(-z))
    return np.divide(1.0, (np.add(1.0, np.exp(np.negative(z)))))

## Implementing Forward Propagation

In [49]:
#predictions = sigmoid(np.ones([m, 1]), sigmoid([np.ones([])]))

# Added bias feature to the input layer
a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

# Activation of the hidden layer
z2 = sigmoid(np.matmul(a1, np.transpose(Theta1)))
      
# Added bias feature to the hidden layer
a2 = np.concatenate([np.ones([m, 1]), z2], axis=1)

# Activation of the output layer
z3 = sigmoid(np.matmul(a2, np.transpose(Theta2)))

# The cost function value:
logErrors = np.multiply(Y, np.log(z3)) + np.multiply(np.subtract(1, Y), np.log(np.subtract(1, z3)))
J = -1/m * np.sum(logErrors)

print(f"The cost value without regularization is {J}")

NameError: name 'Y' is not defined

### Adding the Regularization term

In [48]:
# Choosing the lambda value
lmda = 0.1

# The reg term
reg = lmda/(2*m) * (np.sum((Theta1[:, 1:]**2)) + np.sum((Theta2[:, 1:]**2)))

J = -1/m * np.sum(logErrors) + reg
print(f"The cost value with regularization is {J}")

NameError: name 'logErrors' is not defined

### Implementing the sigmoidGradient function

In [10]:
def sigmoidGradient(z):
    return np.multiply(sigmoid(z), np.subtract(1, sigmoid(z)))

### Randomly initialize the parameters for symmetry breaking

In [11]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = np.sqrt(6)/(np.sqrt(L_in + L_out))
    return np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

## Backward Propagation

In [82]:
# Added bias feature to the input layer
a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

# Matrix Product of a1 and Theta1
z2 = np.matmul(a1, np.transpose(Theta1))

# Activation of the hidden layer
a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

# Matrix Product of a2 and Theta2
z3 = np.matmul(a2, np.transpose(Theta2))

# Activation of the output layer
a3 = sigmoid(z3)

# Error of the output layer
d3 = np.subtract(a3, Y)

# Error of the hidden layer
d2 = np.multiply(np.matmul(d3, Theta2[:, 1:]), sigmoidGradient(z2))

# Accumulate the gradient, vectorized version
Delta1 = np.matmul(np.transpose(d2), a1)
Delta2 = np.matmul(np.transpose(d3), a2)

# Obtain the gradient by dividing the accumulated gradients by m
Theta1_grad = np.divide(Delta1, m)
Theta2_grad = np.divide(Delta2, m)

# Add regularization to the gradient
Theta1_grad[:, 2:] = Theta1_grad[:, 2:] + lmda/m*Theta1_grad[:, 2:]
Theta2_grad[:, 2:] = Theta2_grad[:, 2:] + lmda/m*Theta2_grad[:, 2:]

# Unrolling out gradient parameters
grad = np.concatenate([Theta1_grad.flatten(), Theta2_grad.flatten()])

(10285,)

## Function to find the cost value for scipy.optimize

In [49]:
def costFunction(nn_params, X, y, lmda):
    
    X = np.reshape(X, (-1, 400))
    
    Theta1 = np.reshape(nn_params[0:10025], (-1, 401))
    Theta2 = np.reshape(nn_params[10025:10285], (-1, 26))
    
    m = np.shape(X)[0]
    y[y == 0] = 10
    Y = np.zeros([m, k])
    for i in range(m):
        j = y[i]
        Y[i, j-1] = 1

    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

    z2 = np.matmul(a1, np.transpose(Theta1))

    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

    z3 = np.matmul(a2, np.transpose(Theta2))

    a3 = sigmoid(z3)
    
    logErrors = np.multiply(Y, np.log(a3)) + np.multiply(np.subtract(1, Y), np.log(np.subtract(1, a3)))

    reg = lmda/(2*m) * (np.sum((Theta1[:, 1:]**2)) + np.sum((Theta2[:, 1:]**2)))

    return -1/m * np.sum(logErrors) + reg

## Function to find the grad for scipy.optimize

In [48]:
def backPropagate(nn_params, X, y, lmda):
    
    X = np.reshape(X, (-1, 400))
    
    Theta1 = np.reshape(nn_params[0:10025], (-1, 401))
    Theta2 = np.reshape(nn_params[10025:10285], (-1, 26))
    
    m = np.shape(X)[0]
    y[y == 0] = 10
    Y = np.zeros([m, k])
    for i in range(m):
        j = y[i]
        Y[i, j-1] = 1
        
    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

    z2 = np.matmul(a1, np.transpose(Theta1))

    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

    z3 = np.matmul(a2, np.transpose(Theta2))

    a3 = sigmoid(z3)

    d3 = np.subtract(a3, Y)

    d2 = np.multiply(np.matmul(d3, Theta2[:, 1:]), sigmoidGradient(z2))

    Delta1 = np.matmul(np.transpose(d2), a1)
    Delta2 = np.matmul(np.transpose(d3), a2)

    Theta1_grad = np.divide(Delta1, m)
    Theta2_grad = np.divide(Delta2, m)

    Theta1_grad[:, 2:] = Theta1_grad[:, 2:] + lmda/m*Theta1_grad[:, 2:]
    Theta2_grad[:, 2:] = Theta2_grad[:, 2:] + lmda/m*Theta2_grad[:, 2:]

    return np.concatenate([Theta1_grad.flatten(), Theta2_grad.flatten()])

# scipy.optimize, fmin_cg

### fmin_cg takes my cost_function, gradient_function, params and tries to optimize the cost value by iterating the gradient_function

In [63]:
def fmin_cg_train():
    randomThetas_unrolled = np.concatenate([randInitializeWeights(400, 25).flatten(), randInitializeWeights(25, 10).flatten()])
    return fmin_cg(costFunction, fprime=backPropagate, x0=randomThetas_unrolled, args=(X_train.flatten(), y_train.flatten(), 0.1), maxiter=400, disp=True, full_output=True )

## Creating a Model

In [64]:
learned_Thetas = fmin_cg_train()

         Current function value: 2.316885
         Iterations: 400
         Function evaluations: 709
         Gradient evaluations: 709


## Saving the model

In [65]:
with open('./nn_model.pkl', 'wb') as f:
    pickle.dump(learned_Thetas, f)

### Load the model

In [66]:
with open('./nn_model.pkl', 'rb') as f:
    loaded_Thetas = pickle.load(f)[0]

## Predict

In [25]:
# Shaping Thetas
Theta1 = np.reshape(loaded_Thetas[0:10025], (-1, 401))
Theta2 = np.reshape(loaded_Thetas[10025:10285], (-1, 26))

# Making a prediction
a1 = np.concatenate([np.ones([m, 1]), X], axis=1)
z2 = np.matmul(a1, np.transpose(Theta1))
a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)
z3 = np.matmul(a2, np.transpose(Theta2))
a3 = sigmoid(z3)

# Finding the indeces of the max value in each row
predicted_values = np.argmax(a3, axis=1)

### Implementation of Prediction function

In [53]:
def nn_predict(learned_Thetas, X):
    Theta1 = np.reshape(learned_Thetas[0:10025], (-1, 401))
    Theta2 = np.reshape(learned_Thetas[10025:10285], (-1, 26))

    m = np.shape(X)[0]
    
    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)
    z2 = np.matmul(a1, np.transpose(Theta1))
    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)
    z3 = np.matmul(a2, np.transpose(Theta2))
    a3 = sigmoid(z3)

    return np.add(np.argmax(a3, axis=1), 1)

## Evaluating our model on the test_set

In [67]:
# Making predictions
predictions = nn_predict(loaded_Thetas, X_test)

# Compare them to actual labels and get the accuracy
print(np.mean((predictions == y_test.flatten()) * 100))

#for i in range(1000):
#    print(predictions[i], y_test[i])

8.6


# I am working on accuracy :(